In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

## Read Data

In [ ]:
data = pd.read_csv("amazon_reviews_us_Office_Products_v1_00.tsv", sep='\t', on_bad_lines='skip')

<ipython-input-3-0adcb062aa40>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("amazon_reviews_us_Office_Products_v1_00.tsv", sep='\t', on_bad_lines='skip')


In [ ]:
data

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0.0,0.0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2640249,US,53005790,RLI7EI10S7SN0,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,26.0,26.0,N,N,Great value! A must if you hate to carry thing...,I can't live anymore whithout my Palm III. But...,1998-12-07
2640250,US,52188548,R1F3SRK9MHE6A3,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,18.0,18.0,N,N,Attaches the Palm Pilot like an appendage,Although the Palm Pilot is thin and compact it...,1998-11-30
2640251,US,52090046,R23V0C4NRJL8EM,0807865001,307284585,Gods and Heroes of Ancient Greece,Office Products,4,9.0,16.0,N,N,"Excellent information, pictures and stories, I...",This book had a lot of great content without b...,1998-10-15
2640252,US,52503173,R13ZAE1ATEUC1T,1572313188,870359649,Microsoft EXCEL 97/ Visual Basic Step-by-Step ...,Office Products,5,0.0,0.0,N,N,class text,I am teaching a course in Excel and am using t...,1998-08-22


## Keep Reviews and Ratings

In [ ]:
data['review_body']

,review_body
0,Great product.
1,What's to say about this commodity item except...
2,"Haven't used yet, but I am sure I will like it."
3,Although this was labeled as &#34;new&#34; the...
4,Gorgeous colors and easy to use
...,...
2640249,I can't live anymore whithout my Palm III. But...
2640250,Although the Palm Pilot is thin and compact it...
2640251,This book had a lot of great content without b...
2640252,I am teaching a course in Excel and am using t...


In [ ]:
data['star_rating']

,star_rating
0,5
1,5
2,5
3,1
4,4
...,...
2640249,4
2640250,4
2640251,4
2640252,5


In [ ]:
input_data = data[['review_body', 'star_rating']]
input_data

,review_body,star_rating
0,Great product.,5
1,What's to say about this commodity item except...,5
2,"Haven't used yet, but I am sure I will like it.",5
3,Although this was labeled as &#34;new&#34; the...,1
4,Gorgeous colors and easy to use,4
...,...,...
2640249,I can't live anymore whithout my Palm III. But...,4
2640250,Although the Palm Pilot is thin and compact it...,4
2640251,This book had a lot of great content without b...,4
2640252,I am teaching a course in Excel and am using t...,5


In [ ]:
input_data.head(3)

,review_body,star_rating
0,Great product.,5
1,What's to say about this commodity item except...,5
2,"Haven't used yet, but I am sure I will like it.",5


In [ ]:
input_data = input_data.dropna()

In [ ]:
input_data['star_rating'] = input_data['star_rating'].astype(int)

<ipython-input-10-29a186775883>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['star_rating'] = input_data['star_rating'].astype(int)


In [ ]:
rating_counts = {
    f"reviews with {num} Rating": (input_data['star_rating'] == num).sum()
    for num in range(1, 6)
}

# Print the rating counts
for rating, count in rating_counts.items():
    print(f"{rating}: {count}")

reviews with 1 Rating: 306967
reviews with 2 Rating: 138381
reviews with 3 Rating: 193680
reviews with 4 Rating: 418348
reviews with 5 Rating: 1582704


 ## We form three classes and select 100000 reviews randomly from each class.





In [ ]:
#Positive Sentiment -> 1 and Negative Sentiment -> 0
input_data['star_rating'] = pd.to_numeric(input_data['star_rating'])
input_data['sentiment'] = np.where(input_data['star_rating'] > 3, 1,
                                 np.where(input_data['star_rating'] <= 2, 0, np.nan))


<ipython-input-12-757a10b7ce8e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['star_rating'] = pd.to_numeric(input_data['star_rating'])
<ipython-input-12-757a10b7ce8e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['sentiment'] = np.where(input_data['star_rating'] > 3, 1,


In [ ]:
input_data

,review_body,star_rating,sentiment
0,Great product.,5,1.0
1,What's to say about this commodity item except...,5,1.0
2,"Haven't used yet, but I am sure I will like it.",5,1.0
3,Although this was labeled as &#34;new&#34; the...,1,0.0
4,Gorgeous colors and easy to use,4,1.0
...,...,...,...
2640249,I can't live anymore whithout my Palm III. But...,4,1.0
2640250,Although the Palm Pilot is thin and compact it...,4,1.0
2640251,This book had a lot of great content without b...,4,1.0
2640252,I am teaching a course in Excel and am using t...,5,1.0


In [ ]:
neutral_count = input_data['sentiment'].isna().sum()
print(f"Neutral reviews count: {neutral_count}")

Neutral reviews count: 193680


In [ ]:
positive_count = (input_data['sentiment'] == 1).sum()
print(f"Positive reviews count: {positive_count}")

Positive reviews count: 2001052


In [ ]:
negative_count = (input_data['sentiment'] == 0).sum()
print(f"Negative reviews count: {negative_count}")

Negative reviews count: 445348


In [ ]:
positive_data = input_data[input_data['sentiment'] == 1].sample(n=100000, random_state=40)
negative_data = input_data[input_data['sentiment'] == 0].sample(n=100000, random_state=40)
input_data = pd.concat([positive_data, negative_data])
input_data

,review_body,star_rating,sentiment
972403,Had to trim it a bit to fit the connection in ...,4,1.0
1972071,Ordered these online before I ran out. So much...,5,1.0
2054272,Looked good - purchased for someone who loves ...,5,1.0
189004,"Nice phones, works well. Best cordless that I...",5,1.0
1056264,This is a great improvement over other phones ...,4,1.0
...,...,...,...
506218,The included paper is not good. I thought it w...,2,0.0
2032209,This toner was advertised as HP factory rebuil...,1,0.0
1827132,"I received this, and it didn't work unfortunat...",1,0.0
1347499,The cyan and magenta cartridges leaked all ove...,2,0.0


In [ ]:
total_length_before_cleaning = [len(review) for review in input_data['review_body']]
average_length_before_cleaning = np.mean(total_length_before_cleaning)
average_length_before_cleaning

319.089205

# Data Cleaning



In [ ]:
input_data['review_body'].astype(str)
input_data

,review_body,star_rating,sentiment
972403,Had to trim it a bit to fit the connection in ...,4,1.0
1972071,Ordered these online before I ran out. So much...,5,1.0
2054272,Looked good - purchased for someone who loves ...,5,1.0
189004,"Nice phones, works well. Best cordless that I...",5,1.0
1056264,This is a great improvement over other phones ...,4,1.0
...,...,...,...
506218,The included paper is not good. I thought it w...,2,0.0
2032209,This toner was advertised as HP factory rebuil...,1,0.0
1827132,"I received this, and it didn't work unfortunat...",1,0.0
1347499,The cyan and magenta cartridges leaked all ove...,2,0.0


In [ ]:
#convert the all reviews into the lower case.
lowered_review_body = input_data['review_body'].str.lower()
input_data['review_body'] = lowered_review_body
input_data

,review_body,star_rating,sentiment
972403,had to trim it a bit to fit the connection in ...,4,1.0
1972071,ordered these online before i ran out. so much...,5,1.0
2054272,looked good - purchased for someone who loves ...,5,1.0
189004,"nice phones, works well. best cordless that i...",5,1.0
1056264,this is a great improvement over other phones ...,4,1.0
...,...,...,...
506218,the included paper is not good. i thought it w...,2,0.0
2032209,this toner was advertised as hp factory rebuil...,1,0.0
1827132,"i received this, and it didn't work unfortunat...",1,0.0
1347499,the cyan and magenta cartridges leaked all ove...,2,0.0


In [ ]:
# remove the HTML and URLs from the reviews
input_data['review_body'] = input_data['review_body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
input_data['review_body'] = input_data['review_body'].str.replace(r'http\S+', '', regex=True)
input_data

<ipython-input-21-1f9e5501689b>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  input_data['review_body'] = input_data['review_body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
<ipython-input-21-1f9e5501689b>:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  input_data['review_body'] = input_data['review_body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())


,review_body,star_rating,sentiment
972403,had to trim it a bit to fit the connection in ...,4,1.0
1972071,ordered these online before i ran out. so much...,5,1.0
2054272,looked good - purchased for someone who loves ...,5,1.0
189004,"nice phones, works well. best cordless that i...",5,1.0
1056264,this is a great improvement over other phones ...,4,1.0
...,...,...,...
506218,the included paper is not good. i thought it w...,2,0.0
2032209,this toner was advertised as hp factory rebuil...,1,0.0
1827132,"i received this, and it didn't work unfortunat...",1,0.0
1347499,the cyan and magenta cartridges leaked all ove...,2,0.0


In [ ]:
contractions_dict = {
    "don't": "do not",
    "can't": "cannot",
    "won't": "will not",
    "i'm": "i am",
    "it's": "it is",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "let's": "let us",
    "you'd": "you would",
    "you'll": "you will",
    "you've": "you have",
    "we're": "we are",
    "we'll": "we will",
    "they're": "they are",
    "they've": "they have",
    "he's": "he is",
    "she's": "she is",
    "that's": "that is",
    "there's": "there is",
    "what's": "what is",
    "here's": "here is",
    "who's": "who is",
    "how's": "how is",
    "i've": "i have",
    "i'd": "i would",
    "we'd": "we would",
    "they'd": "they would",
    "it'd": "it would",
    "he'd": "he would",
    "she'd": "she would",
    "doesn't": "does not",
    "didn't": "did not",
    "couldn't": "could not",
    "shouldn't": "should not",
    "mightn't": "might not",
    "mustn't": "must not",
}

In [ ]:
contractions = re.compile('|'.join(re.escape(key) for key in contractions_dict.keys()))
input_data['review_body'] = input_data['review_body'].apply(
    lambda x: contractions.sub(lambda match: contractions_dict[match.group(0)], x)
)


In [ ]:
input_data

,review_body,star_rating,sentiment
972403,had to trim it a bit to fit the connection in ...,4,1.0
1972071,ordered these online before i ran out. so much...,5,1.0
2054272,looked good - purchased for someone who loves ...,5,1.0
189004,"nice phones, works well. best cordless that i...",5,1.0
1056264,this is a great improvement over other phones ...,4,1.0
...,...,...,...
506218,the included paper is not good. i thought it w...,2,0.0
2032209,this toner was advertised as hp factory rebuil...,1,0.0
1827132,"i received this, and it did not work unfortuna...",1,0.0
1347499,the cyan and magenta cartridges leaked all ove...,2,0.0


In [ ]:
#remove non-alphabetical characters
input_data['review_body'] = input_data['review_body'].str.replace(r'[^a-z\s]','', regex=True)
input_data

,review_body,star_rating,sentiment
972403,had to trim it a bit to fit the connection in ...,4,1.0
1972071,ordered these online before i ran out so much ...,5,1.0
2054272,looked good purchased for someone who loves n...,5,1.0
189004,nice phones works well best cordless that i h...,5,1.0
1056264,this is a great improvement over other phones ...,4,1.0
...,...,...,...
506218,the included paper is not good i thought it wo...,2,0.0
2032209,this toner was advertised as hp factory rebuil...,1,0.0
1827132,i received this and it did not work unfortunat...,1,0.0
1347499,the cyan and magenta cartridges leaked all ove...,2,0.0


In [ ]:
# remove extra spaces
input_data['review_body'] = input_data['review_body'].str.replace(' +', ' ', regex=True)
input_data

,review_body,star_rating,sentiment
972403,had to trim it a bit to fit the connection in ...,4,1.0
1972071,ordered these online before i ran out so much ...,5,1.0
2054272,looked good purchased for someone who loves na...,5,1.0
189004,nice phones works well best cordless that i ha...,5,1.0
1056264,this is a great improvement over other phones ...,4,1.0
...,...,...,...
506218,the included paper is not good i thought it wo...,2,0.0
2032209,this toner was advertised as hp factory rebuil...,1,0.0
1827132,i received this and it did not work unfortunat...,1,0.0
1347499,the cyan and magenta cartridges leaked all ove...,2,0.0


In [ ]:
total_length_after_cleaning = [len(review) for review in input_data['review_body']]
average_length_after_cleaning = np.mean(total_length_after_cleaning)
average_length_after_cleaning

303.057435

In [ ]:
input_data.head(3)

,review_body,star_rating,sentiment
972403,had to trim it a bit to fit the connection in ...,4,1.0
1972071,ordered these online before i ran out so much ...,5,1.0
2054272,looked good purchased for someone who loves na...,5,1.0


In [ ]:
total_length_before_processing = [len(review) for review in input_data['review_body']]
average_length_before_processing = np.mean(total_length_before_processing)
average_length_before_processing

303.057435

# Pre-processing

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

## remove the stop words and perform lemmatisation

In [ ]:
input_data

,review_body,star_rating,sentiment
972403,had to trim it a bit to fit the connection in ...,4,1.0
1972071,ordered these online before i ran out so much ...,5,1.0
2054272,looked good purchased for someone who loves na...,5,1.0
189004,nice phones works well best cordless that i ha...,5,1.0
1056264,this is a great improvement over other phones ...,4,1.0
...,...,...,...
506218,the included paper is not good i thought it wo...,2,0.0
2032209,this toner was advertised as hp factory rebuil...,1,0.0
1827132,i received this and it did not work unfortunat...,1,0.0
1347499,the cyan and magenta cartridges leaked all ove...,2,0.0


In [ ]:
def preprocess_text(text):
    if not isinstance(text, str):
        text = ' '.join(text) if isinstance(text, list) else str(text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [ ]:
input_data['review_body'] = input_data['review_body'].astype(str)
input_data['review_body'] = input_data['review_body'].apply(preprocess_text)

In [ ]:
input_data

,review_body,star_rating,sentiment
972403,trim bit fit connection phone got plugged work...,4,1.0
1972071,ordered online ran much nicer running store br...,5,1.0
2054272,looked good purchased someone love nascar sure...,5,1.0
189004,nice phone work well best cordless owned far,5,1.0
1056264,great improvement phone really hear set easy e...,4,1.0
...,...,...,...
506218,included paper good thought would good use rec...,2,0.0
2032209,toner advertised hp factory rebuilt fact brand...,1,0.0
1827132,received work unfortunately disappointed suppo...,1,0.0
1347499,cyan magenta cartridge leaked hand pant opened...,2,0.0


In [ ]:
input_data.head(3)

,review_body,star_rating,sentiment
972403,trim bit fit connection phone got plugged work...,4,1.0
1972071,ordered online ran much nicer running store br...,5,1.0
2054272,looked good purchased someone love nascar sure...,5,1.0


In [ ]:
total_length_after_processing = [len(review) for review in input_data['review_body']]
average_length_after_processing = np.mean(total_length_after_processing)
average_length_after_processing

188.68099

# TF-IDF Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf_idf = TfidfVectorizer()

In [ ]:
result = tf_idf.fit_transform(input_data['review_body'])

##Splitting Dataset into train and test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(result, input_data['sentiment'], test_size=0.2,random_state=40)

In [ ]:
print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")

Training set shape: (160000, 125483), (160000,)
Testing set shape: (40000, 125483), (40000,)


# Perceptron

In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
perceptron = Perceptron(max_iter=1200, eta0=0.1, random_state=40)
perceptron.fit(X_train, y_train)

Perceptron(eta0=0.1, max_iter=1200, random_state=40)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
y_train_pred = perceptron.predict(X_train)
y_test_pred = perceptron.predict(X_test)

In [ ]:
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='binary')
train_recall = recall_score(y_train, y_train_pred, average='binary')
train_f1 = f1_score(y_train, y_train_pred, average='binary')

In [ ]:
print("Training Data:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1-Score: {train_f1:.4f}")

Training Data:
Accuracy: 0.9025
Precision: 0.8968
Recall: 0.9096
F1-Score: 0.9031


In [ ]:
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='binary')
test_recall = recall_score(y_test, y_test_pred, average='binary')
test_f1 = f1_score(y_test, y_test_pred, average='binary')

In [ ]:
print("Testing Data:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {test_f1:.4f}")

Testing Data:
Accuracy: 0.8510
Precision: 0.8475
Recall: 0.8568
F1-Score: 0.8521


# SVM

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
linear_svc = LinearSVC(random_state=40, max_iter=1200)
linear_svc.fit(X_train, y_train)

LinearSVC(max_iter=1200, random_state=40)

In [ ]:
y_train_pred = linear_svc.predict(X_train)
y_test_pred = linear_svc.predict(X_test)

In [ ]:
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='binary')
train_recall = recall_score(y_train, y_train_pred, average='binary')
train_f1 = f1_score(y_train, y_train_pred, average='binary')

In [ ]:
print("Training Metrics:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1-Score: {train_f1:.4f}")

Training Metrics:
Accuracy: 0.9377
Precision: 0.9385
Recall: 0.9369
F1-Score: 0.9377


In [ ]:
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='binary')
test_recall = recall_score(y_test, y_test_pred, average='binary')
test_f1 = f1_score(y_test, y_test_pred, average='binary')

In [ ]:
print("Testing Metrics:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {test_f1:.4f}")

Testing Metrics:
Accuracy: 0.8920
Precision: 0.8947
Recall: 0.8890
F1-Score: 0.8918


# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic = LogisticRegression(random_state=42, max_iter=1200)
logistic.fit(X_train, y_train)

LogisticRegression(max_iter=1200, random_state=42)

In [ ]:
y_train_pred = logistic.predict(X_train)
y_test_pred = logistic.predict(X_test)

In [ ]:
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='binary')
train_recall = recall_score(y_train, y_train_pred, average='binary')
train_f1 = f1_score(y_train, y_train_pred, average='binary')

In [ ]:
print("Training Results:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1-Score: {train_f1:.4f}")

Training Results:
Accuracy: 0.9082
Precision: 0.9114
Recall: 0.9042
F1-Score: 0.9078


In [ ]:
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='binary')
test_recall = recall_score(y_test, y_test_pred, average='binary')
test_f1 = f1_score(y_test, y_test_pred, average='binary')

In [ ]:
print("\nTesting Results:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {test_f1:.4f}")


Testing Results:
Accuracy: 0.8927
Precision: 0.8975
Recall: 0.8870
F1-Score: 0.8922


# Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [ ]:
y_train_pred = nb.predict(X_train)
y_test_pred = nb.predict(X_test)

In [ ]:
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='binary')
train_recall = recall_score(y_train, y_train_pred, average='binary')
train_f1 = f1_score(y_train, y_train_pred, average='binary')

In [ ]:
print("Training Results:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1-Score: {train_f1:.4f}")

Training Results:
Accuracy: 0.8779
Precision: 0.8973
Recall: 0.8534
F1-Score: 0.8748


In [ ]:
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='binary')
test_recall = recall_score(y_test, y_test_pred, average='binary')
test_f1 = f1_score(y_test, y_test_pred, average='binary')

In [ ]:

print("\nTesting Results:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {test_f1:.4f}")


Testing Results:
Accuracy: 0.8548
Precision: 0.8756
Recall: 0.8279
F1-Score: 0.8511
